In [80]:
# from tensorflow.keras import
import chess
from chess import Board, engine
import pandas as pd
import random
import numpy
from stockfish import Stockfish

stockfish = Stockfish("engines/stockfish_15_x64_avx2.exe", depth=20, parameters={"Threads": 8})

In [81]:
from traceback import print_exc
# funkcja zwracajaca punktacje dla danego silnika i boarda
def get_scores_valuation_from_engine(fen) -> float:
    try:
        stockfish.set_fen_position(fen)
        score = stockfish.get_evaluation()
        return score["value"]
    except Exception:
        print_exc()


In [82]:
# this function will create our x (board)
def get_board_from_fen(fen: str):
    board = chess.Board()
    board.set_fen(fen=fen)
    return board

In [83]:


# board -> 3d board dla sieci neuronowej
squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [84]:
%%time
def preprocess_pipeline(file_path: str):
    data = pd.DataFrame(columns=["board3d", "scores"])
    with open(file_path, 'r') as file:
        for single_fen in file.readlines():
            print(single_fen)
            board = get_board_from_fen(single_fen)
            print(board)
            score = get_scores_valuation_from_engine(single_fen)
            print(score)
            board3d = split_dims(board)
            print(board3d)
            # df = pd.DataFrame([board3d, score], columns=["board3d", "scores"])
            # data = pd.concat([data, df], axis=1)
    return data

CPU times: total: 0 ns
Wall time: 0 ns


In [85]:
%%time
preprocess_pipeline("fen_dataset.txt")

Q4b1r/ppp2ppp/4k3/4p3/3P4/8/PPP2PPP/RNBQKBNR b KQ - 0 8
Q . . . . b . r
p p p . . p p p
. . . . k . . .
. . . . p . . .
. . . P . . . .
. . . . . . . .
P P P . . P P P
R N B Q K B N R
3205
[[[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 1 1 0 0 1 1 1]
  [0 0 0 0 0 0 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 1 0 0 0 0 1 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 1 0 0 1 0 0]]

 [[0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 1]]

 [[1 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0]]

 

,board3d,scores
